<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.core.common import SettingWithCopyWarning

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit
from scipy import stats as st
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

В первую очередь загрузим наши данные, проверим наличие пропусков

In [2]:
alpha = pd.read_csv('/content/insurance.csv') 

In [3]:
alpha.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
alpha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
alpha.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Переименуем столбцы на английский язык

In [6]:
alpha = alpha.rename(columns={'Пол': 'Sex', 'Возраст':'Age',
                              'Зарплата': 'Salary', 'Члены семьи': 'Family', 
                              'Страховые выплаты': 'Count'})

In [7]:
alpha.head()

,Sex,Age,Salary,Family,Count
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Пропуски не обнаружены. Для столбцов Возраст и Зарплата используем pd.to_numeric() для преобразования аргумента в числовой тип.

In [8]:
alpha['Age'] = pd.to_numeric(alpha['Age'])
alpha['Salary'] = pd.to_numeric(alpha['Salary'])

## Умножение матриц

Выведем параметры и целевой признак, напишем формулу для модели и поиска метрики r2

In [9]:
features = alpha.drop('Count',axis=1)
target = alpha['Count']

In [10]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4249455028666801


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Значение метрики r2 не изменится

**Обоснование:** 
Заменим нашу матрицу X на рандомную R. Новая матрица - это умножение X на матрицу D. Тогда: 
$$
𝑎1 = R(R^T R)^{-1} R^Ty
$$

$$
𝑎1 = RR^{-1}(R^T)^{-1} R^Ty
$$
 


При умножении обратной матрицы $A^{-1}$ на матрицу $A$ получается единичная матрица $E$ (квадратная матрица, на главной диагонали которой стоят единицы, а остальные элементы — нули). Произведение любой матрицы и единичной матрицы подходящего размера равно самой матрице:
$$
(A^𝑇)^{-1}A^𝑇 = E
$$

Для начала вспомним, что $$(A*B)^T = A^T * B^T$$

Немного изменим формулу, оставим Х матрицу, и умножим ее на радномную обратимую матрицу O

$$
w = ((XO)^T(XO))^{-1}(XO)^Ty
$$

Начнем раскрывать скобки. Начнем с последней

$$
w = ((XO)^T(XO))^{-1}X^TO^Ty
$$

Раскроем первую общую скобку
    
$$
w = (O^TX^T(XO))^{-1}X^TO^Ty
$$

$$
w = O^{-1}(X^TX)^{-1}(O^T)^{-1}O^TX^Ty
$$

При умножении обратной матрицы  𝐴−1  на матрицу  𝐴  получается единичная матрица  𝐸  (квадратная матрица, на главной диагонали которой стоят единицы, а остальные элементы — нули). Произведение любой матрицы и единичной матрицы подходящего размера равно самой матрице:

$$
(A^𝑇)^{-1}A^𝑇 = E
$$

Сократим квадратные матрицы

$$
w = O^{-1}(X^TX)^{-1}X^T
$$

$$
a1 = (OX)w
$$

$$
a1 = (OX)O^{-1}(X^TX)^{-1}X^T 
$$

$$
a1 = X(X^TX)^{-1}X^T
$$

$$
a1 = a
$$

## Алгоритм преобразования

**Алгоритм**
Для проверки нашей теории будем умножать матрицу признаков на обратимую матрицу $RANDOM$, которая будет генерироваться случайным образом. Воспользуемся функциями np.random.rand для создания матрицы 4х4 (такое количесвто нужно для корректого умножения на наши признаки) и np.linalg.inv для создания обратной матрицы

In [11]:
RANDOM = np.random.rand(4,4)
RANDOM2 = np.linalg.inv(RANDOM)

In [12]:
RANDOM

array([[1.01987704e-01, 8.92736381e-01, 4.41662147e-01, 3.30874974e-01],
       [2.63222513e-01, 6.02547798e-01, 6.50925072e-01, 6.20110743e-05],
       [2.45588558e-01, 1.27930825e-01, 1.44128854e-01, 2.25028056e-01],
       [7.03174101e-01, 9.94746728e-01, 1.18656280e-01, 7.69030378e-01]])

In [13]:
RANDOM2

array([[-2.77627005,  1.84817367, -1.08369185,  1.51144191],
       [ 0.04623191,  0.60491826, -3.76396047,  1.0814426 ],
       [ 1.079658  ,  0.22918524,  3.92194503, -1.61215178],
       [ 2.31213747, -2.50773266,  5.25447123, -1.23178017]])

In [14]:
RANDOM @ RANDOM2

array([[ 1.00000000e+00, -4.14724601e-20, -2.10187260e-16,
        -9.89118206e-17],
       [-1.60062392e-16,  1.00000000e+00, -3.25674868e-16,
         1.44566722e-16],
       [-2.87312289e-16, -4.26914843e-17,  1.00000000e+00,
         9.08694806e-17],
       [-1.00883178e-16, -2.81364148e-16, -2.91928485e-16,
         1.00000000e+00]])

In [15]:
nrows = 4
ncols = 4

a = np.random.randint(100, size=(nrows,ncols))
a

array([[66, 57, 10, 98],
       [93, 18, 84,  2],
       [63, 91, 50, 34],
       [18, 38, 48,  1]])

In [16]:
a2 = np.linalg.inv(a)

In [17]:
b  = a @ a2

In [18]:
b 

array([[ 1.00000000e+00, -4.16333634e-17,  1.94289029e-16,
         1.45716772e-16],
       [ 2.08166817e-17,  1.00000000e+00, -1.38777878e-16,
        -1.59594560e-16],
       [-6.24500451e-17, -1.52655666e-16,  1.00000000e+00,
         3.46944695e-17],
       [ 2.42861287e-17, -5.89805982e-17, -3.46944695e-16,
         1.00000000e+00]])

In [19]:
X_inv = features @ RANDOM2

In [20]:
   
model = LinearRegression()
model.fit(X_inv, target)
predictions = model.predict(X_inv)
print(r2_score(target, predictions))


0.424945502866352


## Проверка алгоритма

Для проверки случайныйм образом умножим наши параметры добавим рандомное число и домножим на созданную матрицу

In [21]:
features_new = ((features*25 + 1889) * 15) @ RANDOM2

In [22]:
features_new.head(10)

,0,1,2,3
0,2.010093e+07,4.276845e+06,7.301453e+07,-2.997640e+07
1,1.540554e+07,3.280330e+06,5.594741e+07,-2.296208e+07
2,8.521560e+06,1.816358e+06,3.096704e+07,-1.269105e+07
3,1.690400e+07,3.591713e+06,6.142637e+07,-2.520954e+07
4,1.058535e+07,2.255141e+06,3.846876e+07,-1.577413e+07
5,1.661993e+07,3.537235e+06,6.036540e+07,-2.477687e+07
6,1.609353e+07,3.424600e+06,5.845910e+07,-2.399256e+07
7,1.564966e+07,3.325005e+06,5.686500e+07,-2.333415e+07
8,2.014133e+07,4.284305e+06,7.316866e+07,-3.003889e+07
9,2.095100e+07,4.455286e+06,7.611576e+07,-3.124962e+07


Значения в наших признаках теперь заполнены совершенно иныи числами, проверим изменится ли значение метрики r2

In [23]:
 
model = LinearRegression()
model.fit(features_new, target)
predictions = model.predict(features_new)
print(r2_score(target, predictions))

0.42494550286661503


## Вывод

Мы загрузили данные, проверили на наличие пропусков столбцы. Для корректного использования поменяли названия на английский язык. Из таблицы вывели таргет и параметры. После этого написали функцию для обучения нашей модели. После этого на примере формулы доказали теорию, что при умножении метрик на матрицу результат r2 не должен поменяться. После этого мы провели исследования нескольких обратимых матриц, проверили на практике теорию о том, что если перемножить матрицу с обратной матрицей, то получается единичная матрица. После этого мы количественно изменили наши метрики для проверки гипотезы на практике. В результате значение r2 не поменялось. 

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования